<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

Try id #259

In [1]:
import sys
sys.path.append('..')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [2]:
import taiko as tk
from taiko.play import *
from taiko.performance import *
from taiko.model import *
from taiko.database import *
from taiko.tools.config import *
from taiko.tools.score import *
from taiko.visualize import *

from collections import deque
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import posixpath
from scipy.stats import mode
from skimage.io import imshow, imsave, imread
import multiprocessing
from tqdm import tqdm

sns.set(font_scale=1.5)

Using TensorFlow backend.


In [7]:
record_id = 254
lgbm = LGBM()
record_df = load_record_df()
record_row = record_df.loc[record_id]
play = get_play(record_row)
pf = get_performance(play, id_=record_id)

train_df = pf.copy()
x = train_df.drop('timestamp', axis=1)
ts = train_df['timestamp']
hit_type_x = lgbm.predict(x)
train_df['hit_type'] = hit_type_x

pred_df = pd.DataFrame(data={
    'timestamp': ts,
    'hit_type': hit_type_x
})

pred_ptr = 0
pred_pool = deque([0] * 10)
song_score = 0
hit_count = 0

pred_hit_types = []
true_hit_types = []
for event_time, hit_type in play.events:
    while pred_df.loc[pred_ptr].timestamp <= event_time + 0.24:
        pred_hit_type = int(pred_df.loc[pred_ptr].hit_type)
        pred_hit_type = transform_hit_type(pred_hit_type)
        pred_pool.popleft()
        pred_pool.append(pred_hit_type)
        pred_ptr += 1

    pool_hit_type = mode(pred_pool)[0]
    print(pred_pool)
    print(pool_hit_type, hit_type)
    multiplier = get_gained_score_multiplier(pool_hit_type, hit_type)
    if multiplier > 0:
        hit_count += 1
        song_score += SCORE_UNIT_DICT[1] * multiplier

print(song_score, hit_count)

deque([2, 1, 2, 2, 1, 1, 1, 1, 1, 0])
[1] 1
deque([1, 1, 1, 1, 0, 0, 0, 0, 0, 0])
[0] 0
deque([0, 0, 0, 0, 0, 0, 0, 0, 2, 2])
[0] 0
deque([0, 0, 0, 2, 2, 2, 2, 1, 2, 1])
[2] 0
deque([2, 2, 2, 1, 2, 1, 1, 2, 1, 1])
[1] 1
deque([1, 1, 2, 1, 1, 1, 0, 0, 0, 0])
[1] 0
deque([1, 0, 0, 0, 0, 0, 0, 0, 2, 2])
[0] 0
deque([0, 0, 0, 2, 2, 2, 2, 2, 1, 1])
[2] 0
deque([2, 2, 2, 1, 1, 1, 1, 1, 3, 3])
[1] 1
deque([1, 1, 1, 3, 3, 3, 1, 1, 2, 2])
[1] 0
deque([3, 1, 1, 2, 2, 1, 1, 1, 1, 3])
[1] 1
deque([1, 1, 1, 1, 3, 3, 0, 0, 0, 2])
[1] 0
deque([3, 3, 0, 0, 0, 2, 1, 1, 2, 2])
[0] 1
deque([2, 1, 1, 2, 2, 2, 2, 0, 0, 0])
[2] 0
deque([2, 2, 0, 0, 0, 0, 0, 0, 0, 1])
[0] 0
deque([0, 0, 0, 0, 1, 1, 2, 1, 2, 1])
[0] 0
deque([1, 2, 1, 2, 1, 1, 1, 1, 1, 1])
[1] 1
deque([1, 1, 1, 1, 1, 1, 0, 0, 0, 0])
[1] 0
deque([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
[0] 0
deque([0, 0, 0, 0, 0, 2, 2, 2, 2, 1])
[0] 0
deque([0, 2, 2, 2, 2, 1, 1, 1, 2, 2])
[2] 1
deque([1, 1, 1, 2, 2, 1, 0, 0, 0, 0])
[0] 0
deque([1, 0, 0, 0, 0, 0, 0, 0, 0

[3] 1
deque([1, 1, 1, 2, 3, 3, 2, 1, 1, 2])
[1] 0
deque([3, 2, 1, 1, 2, 1, 1, 2, 1, 1])
[1] 1
deque([1, 1, 2, 1, 1, 1, 0, 0, 0, 0])
[1] 0
deque([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
[0] 0
deque([0, 0, 0, 0, 0, 1, 3, 3, 1, 5])
[0] 0
deque([1, 3, 3, 1, 5, 5, 5, 3, 4, 3])
[3] 3
deque([5, 5, 3, 4, 3, 1, 0, 0, 0, 0])
[0] 0
deque([3, 1, 0, 0, 0, 0, 0, 0, 0, 0])
[0] 0
deque([0, 0, 0, 0, 0, 3, 2, 1, 1, 2])
[0] 0
deque([3, 2, 1, 1, 2, 5, 5, 3, 4, 1])
[1] 3
deque([5, 5, 3, 4, 1, 1, 0, 0, 0, 0])
[0] 0
deque([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
[0] 0
deque([0, 0, 0, 0, 0, 2, 2, 1, 3, 3])
[0] 0
deque([2, 2, 1, 3, 3, 5, 5, 5, 5, 5])
[5] 6
deque([5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
[5] 6
deque([5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
[5] 6
deque([5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
[5] 6
deque([5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
[5] 6
deque([5, 5, 5, 5, 5, 5, 5, 5, 5, 5])
[5] 0
deque([5, 5, 5, 5, 5, 3, 3, 3, 0, 3])
[5] 0
deque([3, 3, 3, 0, 3, 0, 3, 1, 1, 1])
[3] 0
deque([0, 3, 1, 1, 1, 1, 1, 1, 1, 3])
[1] 1
deque([1, 1, 1, 1, 3, 0, 0

In [3]:
record_df = load_record_df(song_id=1)
# record_df = record_df[(record_df['song_id'] >= 1) & (record_df['song_id'] <= 4)]
record_df

,drummer_name,song_id,left_sensor_datetime,right_sensor_datetime,capture_datetime
1,aaaaa,1,L_2018-09-28_111304.csv,R_2018-09-28_111305.csv,capture_2018_09_28_11_13_05
5,aaaaa,1,L_2018-09-28_112912.csv,R_2018-09-28_112913.csv,capture_2018_09_28_11_29_13
9,aaaaa,1,L_2018-09-28_114340.csv,R_2018-09-28_114340.csv,capture_2018_09_28_11_43_41
13,aaaaa,1,L_2018-09-28_115822.csv,R_2018-09-28_115823.csv,capture_2018_09_28_11_58_23
25,carolyn,1,L_2018-09-27_154913.csv,R_2018-09-27_154916.csv,capture_2018_09_27_15_49_16
29,carolyn,1,L_2018-09-27_160509.csv,R_2018-09-27_160510.csv,capture_2018_09_27_16_05_10
33,carolyn,1,L_2018-09-27_161825.csv,R_2018-09-27_161826.csv,capture_2018_09_27_16_18_26
37,carolyn,1,L_2018-09-27_163110.csv,R_2018-09-27_163111.csv,capture_2018_09_27_16_31_11
42,cuxi,1,L_2018-09-25_155107.csv,R_2018-09-25_155108.csv,capture_2018_09_25_15_51_09
46,cuxi,1,L_2018-09-25_160510.csv,R_2018-09-25_160512.csv,capture_2018_09_25_16_05_13


In [ ]:
lgbm = LGBM()
for id_ in reversed(record_df.index):
    try:
        print(id_, predict_score(id_, lgbm))
    except Exception:
        print(id_, 'error!!!')
# with multiprocessing.Pool(1) as p:
#     for id_, _ in tqdm(enumerate(p.imap(predict_score, record_df.index)), total=len(record_df.index)):
#         pass

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
353 (545480, 44)
349 (597930, 50)
345 (514010, 42)
341 (241270, 18)
336 (409110, 34)
332 (576950, 47)
328 (440580, 39)
324 (367150, 32)
319 (461560, 41)
315 (451070, 37)
311 (618910, 51)
307 (576950, 47)
302 (398620, 36)
298 (713320, 60)
294 (692340, 58)
290 (461560, 41)
283 error!!!
279 (545480, 49)
275 (692340, 62)
271 (524500, 45)
266 (409110, 33)
262 (618910, 51)
258 (367150, 29)
254 (807730, 70)
233 (482540, 42)
232 (472050, 39)
231 (430090, 35)
230 (430090, 37)
226 (545480, 46)
222 (566460, 48)
218 (587440, 53)
214 (786750, 68)
208 (346170, 32)
204 (335680, 29)
200 (482540, 45)
196 (419600, 37)
191 (692340, 60)
187 (304210, 25)
183 (220290, 17)
179 (409110, 34)
177 (115390, 8)
170 (199310, 13)
166 (125880, 8)
162 (83920, 6)
157 (157350, 15)
153 (335680, 27)
